# Preprocessing, GridSearch and Training SpiderNet-8 on Open Data

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/
D:/Share/safanasev/Python-notebook/AF_ML_chinese/


In [3]:
for DIR in ['Data Sets', 'Documentation', 'Graphs', 'Modeling', 'SandBox', 'Selection', 'Statistics']:
    PATH_DIR = PATH + DIR
    print(PATH_DIR)
    pathlib.Path(PATH_DIR).mkdir(parents=True, exist_ok=True)
    
for DIR in ['csv_from_python', 'excel', 'python']:
    PATH_DIR = PATH + 'Selection/' + DIR
    print(PATH_DIR)
    pathlib.Path(PATH_DIR).mkdir(parents=True, exist_ok=True) 

D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Data Sets
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Documentation
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Graphs
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Modeling
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/SandBox
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Selection
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Statistics
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Selection/csv_from_python
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Selection/excel
D:/Share/asmirnova5/Python-notebook/AF_ML_chinese/Selection/python


In [6]:
task = 'binary'
missing_strings = 'MISSING'
p_value = 0.05
target_dict = {'good': 0, 'bad': 1}

In [7]:
target = 'BAD_FLAG'
index_month = 'MONTH_YEAR'
list_of_vars_for_strat = ['MONTH_YEAR']
sort_by_var = 'APPPOSID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0', 'PERIOD_7', 'LOSS_90P'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [8]:
COL_DEL

['UNNAMED: 0', 'PERIOD_7', 'LOSS_90P']

In [9]:
necessary_fields

['BAD_FLAG', 'MONTH_YEAR', 'APPPOSID']

In [10]:
n_jobs = 3

## Start of selection¶

In [11]:
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [12]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [13]:
from keras.utils import np_utils
from tensorflow.keras.models import Model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler
#from keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [15]:
import tensorflow as tf

In [16]:
print(tf.__version__)

2.3.0


In [17]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping

## Loading Data

In [19]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
#загружаем данные для 163 переменных

train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [20]:
y_train = train_for[target]
#y_test = valid_for[target]
#y_val = test_for[target]

y_test = test_for[target]
y_val = valid_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [21]:
#проверка корректности
X_1_2.shape, train_for.shape, len(y_train)

((1507599, 163), (1507599, 164), 1507599)

In [22]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [23]:
#проверка корректности, должно быть везде True
print(X_2_2.shape[0] == valid_for.shape[0])
print(X_2_2.shape[1] == valid_for.shape[1] - 1)
print(len(y_val) == valid_for.shape[0])

True
True
True


In [24]:
#если alpha не задано, то не делаем br_correction
#новая версия, исправлены индексы

class DataGenerator_new(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in indexes]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [25]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
#validation_generator = DataGenerator(valid_for, x_col, y_col, batch_size=valid_for.shape[0], alpha = None, class_w = None)

In [26]:
Y_test_2 = np_utils.to_categorical( y_val, 2) # преобразовываем в 2 класса 

In [27]:
X_2_2.shape, Y_test_2.shape

((185400, 163), (185400, 2))

# *

In [28]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# ***

In [29]:
from tensorflow.keras.layers import GlobalMaxPooling1D

In [30]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

In [31]:
import os
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'

In [32]:
#training_aug = DataGenerator(aug_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )

In [33]:
#learning_rate=0.005
initial_learning_rate = 0.005

def lr_step_decay(epoch, lr):
    drop_rate = 0.1
    epochs_drop = 20.0
    
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [34]:
import urllib
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

In [35]:
from tensorflow.keras.models import load_model

## GRID SEARCH

## SpiderNet-8

In [36]:
def max_pool(min_pool, n_pool, padding_pool, str_pool, y ):
    if y.shape[1] < (min_pool):
        
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=1)(y)
    else:
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_pool)(y)

In [37]:
def spider_model_8block(l2_conv = None, reg = 1E-5 , reg_dense = 1E-5,
                 _bias = True,  n_features = 163, n_pool = 2, n_kernel = 5, n_filters = 5, n_strides = 1,
                 classes = 2, hidden = 64 , drop_out = 0.25, drop_out_conv = 0.001, drop_out_rate = 4 ,
                 padding_pool = 'valid' ,
                gl_pool_max = False,
                       min_pool = 12, str_pool = None):
    
    if reg == None:
        l2_batch_gamma = None 
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta =l2(reg)
    
    if reg_dense == None:
        kernel_regularizer = None
    else:
        kernel_regularizer = l2(reg)   
    
    
    x = Input(shape=(  n_features, 1))
    n = -2 # для 6 - n = 0, для 8 = -2
    y = Conv1D(filters=n_filters, kernel_size=n_kernel, strides=n_strides, padding='same', 
           use_bias=_bias, kernel_regularizer=l2_conv)(x) 

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
#y = Dropout(rate=drop_rate)(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut1_2 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)## поправить
    shortcut1_3 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_2)##
    shortcut1_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_3)##
    
    shortcut1_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_4)
    shortcut1_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_5) #*
    shortcut1_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut1_6) #*

    # второй spider-block
    y = Conv1D(filters=n_filters*2, kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut2_3 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    shortcut2_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_3)##
    
    shortcut2_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_4)
    shortcut2_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_5)
    shortcut2_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut2_6)


    y= Concatenate(axis=-1)([shortcut1_2, y])
    n = n+1

    # третий spider-block
    y = Conv1D(filters=n_filters*(3 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut3_4 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    
    shortcut3_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_4)##
    shortcut3_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_5)##
    shortcut3_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut3_6)##

    y= Concatenate(axis=-1)([shortcut2_3, shortcut1_3, y])
    
    n = n+1
    # четвертый spider-block
    y = Conv1D(filters=n_filters*(4 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut4_5 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)##
    shortcut4_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut4_5)##
    shortcut4_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut4_6)##

    y= Concatenate(axis=-1)([shortcut3_4, shortcut2_4, shortcut1_4, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # пятый spider-block
    y = Conv1D(filters=n_filters*(5 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)

    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut5_6 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)
    shortcut5_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, shortcut5_6)    
    
    
    y= Concatenate(axis=-1)([shortcut4_5, shortcut3_5, shortcut2_5, shortcut1_5, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))

    # шестой spider-block
    y = Conv1D(filters=n_filters*(6 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    shortcut6_7 = max_pool(min_pool, n_pool, padding_pool, str_pool, y)
    
    y= Concatenate(axis=-1)([shortcut5_6, shortcut4_6, shortcut3_6, shortcut2_6, shortcut1_6, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))
    
    
    # седьмой блок
    y = Conv1D(filters=n_filters*(7 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    
    y= Concatenate(axis=-1)([shortcut6_7, shortcut5_7, shortcut4_7, shortcut3_7, shortcut2_7, shortcut1_7, y])
    n = n+1
    y = Dropout(drop_out_conv*(n**drop_out_rate))(y)
    print(n, drop_out_conv*(n**drop_out_rate))
    
     # восьмой блок
    y = Conv1D(filters=n_filters*(8 + 1), kernel_size=n_kernel, strides=n_strides, padding='same',
              use_bias=_bias, kernel_regularizer=l2_conv)(y) 
    y = BatchNormalization(gamma_regularizer=l2_batch_gamma ,beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    
    y = max_pool(min_pool, n_pool, padding_pool, str_pool, y) ##
    
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
    #z = Flatten()(y) #сглаживание, пример использования - https://stackoverflow.com/questions/43237124/what-is-the-role-of-flatten-in-keras
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    #model = Sequential()
    model_15 = Model(inputs=x, outputs=predictions)
    
    return model_15

In [48]:
gl_p = [False] #0
l2_batch = [ 0.0002] #1
n_ker = [5, 7] #2
n_fil = [ 5, 15] #3
d_hidden = [60, 100] #4
drop_out = [0.25] #5
drop_out_conv = [0.001] #6
reg_dense = [ 0.0002] #7
min_pool = [ 15] #7

In [1]:
param = (False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)

In [49]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, drop_out_conv, reg_dense, min_pool)

In [50]:
i = 0
for p in param:
    print(i, p)
    i += 1

0 (False, 0.0002, 5, 5, 60, 0.25, 0.001, 0.0002, 15)
1 (False, 0.0002, 5, 5, 100, 0.25, 0.001, 0.0002, 15)
2 (False, 0.0002, 5, 15, 60, 0.25, 0.001, 0.0002, 15)
3 (False, 0.0002, 5, 15, 100, 0.25, 0.001, 0.0002, 15)
4 (False, 0.0002, 7, 5, 60, 0.25, 0.001, 0.0002, 15)
5 (False, 0.0002, 7, 5, 100, 0.25, 0.001, 0.0002, 15)
6 (False, 0.0002, 7, 15, 60, 0.25, 0.001, 0.0002, 15)
7 (False, 0.0002, 7, 15, 100, 0.25, 0.001, 0.0002, 15)


In [ ]:
(False, 0.0002, 5, 15, 100, 0.25, 0.001, 0.0002, 15)
163 (185400, 163) (False, 0.0002, 7, 15, 100, 0.25, 0.001, 0.0002, 15)

In [51]:
param = (  (False, 0.0002, 5, 5, 64, 0.25, 0.001, 0.0002, 15) #internal spider6 best
         , (False, 0.0002, 3, 10, 100, 0.25, 0.001, 0.0002, 15) #internal cnn best
         
, (False, 0.0002, 7, 15, 100, 0.25, 0.001, 0.0002, 15) ##
        , (False, 0.0002, 7, 15, 60, 0.25, 0.001, 0.0002, 15)
        , (False, 0.0002, 7, 5, 100, 0.25, 0.001, 0.0002, 15)
        , (False, 0.0002, 7, 5, 60, 0.25, 0.001, 0.0002, 15)
        , (False, 0.0002, 5, 15, 100, 0.25, 0.001, 0.0002, 15)) ##

In [43]:
param = ((False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20) ,
        (False, 0.0002, 7, 15, 64, 0.25, 0.001, 0.0002, 20) ,
        (False, 0.0002, 7, 5, 32, 0.25, 0.001, 0.0002, 20))

In [42]:
inp_shape = X_2_2.shape[1]
inp_shape

163

In [44]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0:0.05 , 1:1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)
    
    model_grid = spider_model_8block(reg = p[1], reg_dense = p[7], n_features = inp_shape, n_pool = 2, n_kernel = p[2], 
                              n_filters = p[3], n_strides = 1, classes = 2, 
                              hidden = p[4] , drop_out = p[5], drop_out_conv = p[6], drop_out_rate = 4 ,
                 padding_pool = 'valid' , gl_pool_max = p[0], min_pool = p[8], str_pool = None)
    
    model_grid.compile(loss='categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    
    history_XX = model_grid.fit_generator(generator=training_generator,  #training_aug,
                    validation_data= (X_2_2, Y_test_2) , #Y_test_2 = np_utils.to_categorical( y_val, 2) 
                                     epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                     callbacks=[_time , EarlyStopping(monitor='val_loss', patience=20),
                                               Metrics(valid_data=(X_2_2, Y_test_2))])
    
    res_model_ = pd.DataFrame(history_XX.history, columns = history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 + datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_spider8_internal_grid_' + str(p[8]) + '_' + str(p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[7]) + '_' + str(dd)
    
    model_grid.save( name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')
    
    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1
    
    result_all_8.at[j , 'name_model'] = name_m
    result_all_8.at[j ,'params'] = str(p)
    result_all_8.at[j ,'val_GINI'] = GINI
    result_all_8.at[j ,'val_APS'] = APS
    result_all_8.at[j ,'test_GINI'] = GINI_t
    result_all_8.at[j ,'test_APS'] = APS_t
    
    result_all_8.to_csv('internal_grid_result_all_8_202105_14.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t) 

163 (185400, 163) (False, 0.0002, 7, 5, 64, 0.25, 0.001, 0.0002, 20)
1 0.001
2 0.016
3 0.081
4 0.256
Instructions for updating:
Please use Model.fit, which supports generators.
train_begin | time:  14.05.2021 11:23:00
Epoch 1/150
epoch_end | time:  14.05.2021 11:44:56
 — val_aps:  0.045093 — val_a: 0.920695
2945/2945 - 1345s - loss: 0.0144 - accuracy: 0.9782 - auc: 0.9952 - precision: 0.9782 - recall: 0.9782 - val_loss: 0.2667 - val_accuracy: 0.9474 - val_auc: 0.9708 - val_precision: 0.9474 - val_recall: 0.9474
Epoch 2/150
epoch_end | time:  14.05.2021 12:06:51
 — val_aps:  0.030027 — val_a: 0.863341
2945/2945 - 1317s - loss: 0.0070 - accuracy: 0.9873 - auc: 0.9978 - precision: 0.9873 - recall: 0.9873 - val_loss: 1.0949 - val_accuracy: 0.6947 - val_auc: 0.7138 - val_precision: 0.6947 - val_recall: 0.6947
Epoch 3/150
epoch_end | time:  14.05.2021 12:30:16
 — val_aps:  0.045203 — val_a: 0.926760
2945/2945 - 1402s - loss: 0.0063 - accuracy: 0.9876 - auc: 0.9982 - precision: 0.9876 - recal

Epoch 27/150
epoch_end | time:  14.05.2021 22:43:49
 — val_aps:  0.049221 — val_a: 0.922554
2945/2945 - 1436s - loss: 0.0038 - accuracy: 0.9903 - auc: 0.9992 - precision: 0.9903 - recall: 0.9903 - val_loss: 0.0800 - val_accuracy: 0.9769 - val_auc: 0.9947 - val_precision: 0.9769 - val_recall: 0.9769
Epoch 28/150
epoch_end | time:  14.05.2021 23:08:16
 — val_aps:  0.049662 — val_a: 0.911226
2945/2945 - 1479s - loss: 0.0038 - accuracy: 0.9904 - auc: 0.9992 - precision: 0.9904 - recall: 0.9904 - val_loss: 0.1256 - val_accuracy: 0.9607 - val_auc: 0.9898 - val_precision: 0.9607 - val_recall: 0.9607
Epoch 29/150
epoch_end | time:  14.05.2021 23:33:29
 — val_aps:  0.055740 — val_a: 0.924194
2945/2945 - 1507s - loss: 0.0037 - accuracy: 0.9906 - auc: 0.9993 - precision: 0.9906 - recall: 0.9906 - val_loss: 0.0566 - val_accuracy: 0.9737 - val_auc: 0.9974 - val_precision: 0.9737 - val_recall: 0.9737
Epoch 30/150
epoch_end | time:  14.05.2021 23:57:51
 — val_aps:  0.067841 — val_a: 0.917448
2945/294

Epoch 17/150
epoch_end | time:  16.05.2021 09:21:30
 — val_aps:  0.059671 — val_a: 0.939372
2945/2945 - 6074s - loss: 0.0036 - accuracy: 0.9907 - auc_1: 0.9993 - precision_1: 0.9907 - recall_1: 0.9907 - val_loss: 0.0474 - val_accuracy: 0.9851 - val_auc_1: 0.9980 - val_precision_1: 0.9851 - val_recall_1: 0.9851
Epoch 18/150
epoch_end | time:  16.05.2021 11:02:12
 — val_aps:  0.067503 — val_a: 0.932378
2945/2945 - 6041s - loss: 0.0036 - accuracy: 0.9905 - auc_1: 0.9993 - precision_1: 0.9905 - recall_1: 0.9905 - val_loss: 0.0405 - val_accuracy: 0.9823 - val_auc_1: 0.9986 - val_precision_1: 0.9823 - val_recall_1: 0.9823
Epoch 19/150
epoch_end | time:  16.05.2021 12:43:33
 — val_aps:  0.059075 — val_a: 0.939470
2945/2945 - 6079s - loss: 0.0035 - accuracy: 0.9908 - auc_1: 0.9994 - precision_1: 0.9908 - recall_1: 0.9908 - val_loss: 0.0355 - val_accuracy: 0.9884 - val_auc_1: 0.9986 - val_precision_1: 0.9884 - val_recall_1: 0.9884
Epoch 20/150
epoch_end | time:  16.05.2021 14:24:28
 — val_aps: 

Epoch 8/150
epoch_end | time:  17.05.2021 17:43:22
 — val_aps:  0.063308 — val_a: 0.917641
2945/2945 - 1524s - loss: 0.0052 - accuracy: 0.9885 - auc_2: 0.9989 - precision_2: 0.9885 - recall_2: 0.9885 - val_loss: 0.0331 - val_accuracy: 0.9875 - val_auc_2: 0.9990 - val_precision_2: 0.9875 - val_recall_2: 0.9875
Epoch 9/150
epoch_end | time:  17.05.2021 18:06:49
 — val_aps:  0.035320 — val_a: 0.906066
2945/2945 - 1429s - loss: 0.0050 - accuracy: 0.9888 - auc_2: 0.9990 - precision_2: 0.9888 - recall_2: 0.9888 - val_loss: 0.1275 - val_accuracy: 0.9620 - val_auc_2: 0.9892 - val_precision_2: 0.9620 - val_recall_2: 0.9620
Epoch 10/150
epoch_end | time:  17.05.2021 18:32:31
 — val_aps:  0.068661 — val_a: 0.933709
2945/2945 - 1527s - loss: 0.0049 - accuracy: 0.9890 - auc_2: 0.9990 - precision_2: 0.9890 - recall_2: 0.9890 - val_loss: 0.0557 - val_accuracy: 0.9873 - val_auc_2: 0.9988 - val_precision_2: 0.9873 - val_recall_2: 0.9873
Epoch 11/150
epoch_end | time:  17.05.2021 18:56:48
 — val_aps:  0